<a href="https://colab.research.google.com/github/aleksandr329/SCAN_project/blob/Anna_branch/SCAN_%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%B8%D0%B5_%D1%84%D0%B0%D0%B9%D0%BB%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
import pandas as pd
import json
from pprint import pprint
import re
from collections import Counter

# Открытие датасета.

In [2]:
# для открытия и чтения файла с данным c Google.Drive через Google.Colab

from google.colab import drive
drive.mount('/content/drive')
path_DF = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/Queries.txt'

Mounted at /content/drive


In [3]:
# В файле Queries.txt приведено множество поисковых запросов клиентов продукта СКАН
# (формат: UserId Query - разделитель табуляция).

data = pd.read_csv(path_DF, sep='\t', header=None, names=['UserId', 'Query'])
data.head()

,UserId,Query
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC..."
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(..."
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3..."
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28371 entries, 0 to 28370
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UserId  28371 non-null  object
 1   Query   28371 non-null  object
dtypes: object(2)
memory usage: 443.4+ KB


In [5]:
data.describe(include=object)

,UserId,Query
count,28371,28371
unique,1085,27593
top,00000000-0000-0000-0000-000000000000,"((Exact(Text(""9ff8d3d3-7624-490e-acd8-bfb2e3cf..."
freq,2219,7


In [6]:
print(f'Найдено дубликатов: {data.duplicated().sum()}')
print(f'Найдено пропусков в данных:\n {data.isna().sum()}')

Найдено дубликатов: 0
Найдено пропусков в данных:
 UserId    0
Query     0
dtype: int64


# Открытие json.

In [7]:
from google.colab import drive
drive.mount('/content/drive')
path_JSON = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/ThemesWordClouds.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:

with open(path_JSON) as f:                     # Открываем файл и связываем его с объектом "f"
  words_clouds_data = json.load(f)                           # Загружаем содержимое открытого файла в переменную words_clouds_data

# Открываем как датафрейм
#wc_data = pd.DataFrame(words_clouds_data)

pprint(words_clouds_data[15]['WordCloud'])   #Ключей 572 (от 0 до 571)

['займы',
 'кредиты',
 'кредит',
 'заем',
 'ссуда',
 'кредитный',
 'кредитование',
 'программа',
 'линия',
 'предложение',
 'пакет',
 'портфель',
 'договор',
 'соглашение',
 'фабрика',
 'сервис',
 'сумма',
 'объем',
 'получение',
 'кредита',
 'получить',
 'привлечь',
 'банки',
 'банк']


In [9]:
wc_data = pd.DataFrame(words_clouds_data)
wc_data.head()

,Theme,WordCloud
0,"Письма, указы, заявления от госорганов","[письмо, указ, поправка, заявление, сообщение,..."
1,Истец,"[истец, после, основании, иск, иск, суд, требо..."
2,Ответчик,"[против, ответчик, иск, действие, поведение, с..."
3,Преступник,"[преступник, уличить, уличать, разыскивать, об..."
4,Потерпевшая сторона,"[потерпевшая, сторона, покушение, нападение, о..."


In [248]:
print(f'Количество уникальных тематик в облаках слов: {wc_data.Theme.nunique()}')

Количество уникальных тематик в облаках слов: 572


# Прочее.

In [ ]:
# Фразовые запросы

phrase_queries = ['OrderedText', 'Text', 'OrderedWildcard', 'Wildcard',
                  'OrderedTextAnyDistance', 'InSameSentence', 'InSameParagraph',
                  'Exact']

# Запросы объектов и размерных величин
mesure_queries = ['Id', 'Metadata', 'Name', 'Org', 'Location', 'Trademark',
                  'IndustrialSector', 'IndustrialSectorPositiveTonality',
                  'IndustrialSectorNegativeTonality', 'NewsCluster', 'Date',
                  'Dim', 'AuthorName']

# Запросы фактов, рубрик и прямой речи
fact_queries = ['Speech', 'Event', 'Subject', 'EventType']

# Запросы источников
source_queries = ['Source', 'SourceLevel', 'SourceCategory', 'SourceRegion', 'SourceTopic']

# Остальное
others_queries = ['Language', 'DocumentType']

# Извлечение текста запроса (в работе).

In [165]:
df = data.copy()

In [166]:
# удаляем скобки и заменяем пробелами
df['Query'] = df['Query'].apply(lambda query: query.replace('(', ' '))
df['Query'] = df['Query'].apply(lambda query: query.replace(')', ' '))

In [167]:
# выделение текста запроса через регулярку. Возможно, ее можно доработать.
# не организации, не персоны, не локации = ['Org', 'Name', 'Location']

def get_txt(query):
  query.split(' ')
  pattern = r'["]\S+\s*\S+\s*\S+["]'       #до 3 слов
  result = re.findall(pattern, query)
  return result


df['Text'] = df['Query'].apply(lambda query: get_txt(query)).astype(str)


In [168]:
df['Text'] = df['Text'].apply(lambda x: x.split(','))             # делим строку по запятой
df = df.explode('Text', ignore_index=True)                        # переносим текст на новую строку
df['Text'] = df['Text'].astype(str)

In [169]:
def edit_text(text):
  '''Функция, удаляющая знаки, которые не входят в состав фразы '''
  del_symbols = ['[', ']', '"', "'", '”', '“', '#', '*', '•t', '•txxx']
  for symbol in del_symbols:
    text = text.replace(symbol, '')
  return text

df['Text'] = df['Text'].apply(lambda text: edit_text(text))

In [171]:
 # удаляем строки с пустыми запросами
 del_index = list(df[df.Text == ''].index)
 df = df.drop(index=del_index)

In [246]:
df.sample()

,UserId,Query,Text
102217,2c208fd3-45ba-e811-81ed-9fe8589abf9b,"Text ""жилищное строительство"" OR Text ""...",развитие инфраструктуры


Выделить группы похожих запросов у разных пользователей (по UserId)

In [202]:
df.groupby(['Text'])['UserId'].count().sort_values(ascending=False).head(10)

Text
 ПМЭФ                    1425
 ВТБ                      640
 Газпром                  625
 ММВБ                     603
 законопроект             600
 IMOEX                    581
MOEX                      580
 ММВБ-РТС                 578
 ПАО Московская Биржа     578
 7702077840               576
Name: UserId, dtype: int64

Для выделенных групп запросов по "облакам слов" выработать метрику, максимизирующую качество маппинга некоторых из выделенных облаков на предоставленные тематики.

In [245]:
# проверим первые 1000 популярных запросов
count = 1000

# сставим список популярных слов в запросе
data = list(df.groupby(['Text'])['UserId'].count().sort_values(ascending=False).index[0:count-1])
popular_queries = pd.DataFrame(
    data,
    columns=['Text']
    )

popular_queries = popular_queries.Text.apply(lambda text: text.split())         #разобьем по словам
popular_queries = popular_queries.explode()                                     # перенесем каждое новое слово
popular_queries = list(popular_queries)

# создадим счетчики найденных сопадений
in_clouds = Counter()
not_found = Counter()

# переводим слова в нижний регистр и ищем совпадения
for i in range(0, len(popular_queries)):
  for j in range(0, 572):
    if popular_queries[i].lower() in words_clouds_data[j]['WordCloud']:
      in_clouds[popular_queries[i]] += 1
    else:
      not_found[popular_queries[i].lower()] += 1

print(f'Найдено совпадений {len(in_clouds)} из {len(in_clouds)+len(not_found)}.')

Найдено совпадений 383 из 1499.
